# Hello Guys!!
# Welcome!!
# To start of, lets begin with same basic info

In [ ]:
##Initializing all libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
##reading the dataframe
df = pd.read_csv("/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv")
df.head()

In [ ]:
#checking for datatypes
df.dtypes

In [ ]:
#finding the number of unique values
df['sex'].unique()

In [ ]:
#find the number of unique values for output dataframe
df['output'].unique()

In [ ]:
#finding if the dataframe is empty or is there any null values
df.isnull().sum()

In [ ]:
#checking for the shape of the dataframe
df.shape

In [ ]:
#Find the duplicates in the dataframe
df.loc[df.duplicated(keep=False),:]

In [ ]:
#droping duplicates
df=df.drop_duplicates()

In [ ]:
#now seeing the shape
df.shape

In [ ]:
#Now checking for correlation
plt.figure(figsize=(25,15))
sns.heatmap(df.corr(),annot=True)

# Now lets do some EDA
# 
# 

In [ ]:
plt.figure(figsize=(4,5))
plt.pie(x=df['sex'].value_counts(),labels=df['sex'].value_counts().index,autopct="%1.1f%%")
plt.title("Ratio Gender")
plt.show()

In [ ]:
plt.figure(figsize=(5,3))
sns.countplot(x='sex',hue='output',data=df)
plt.title("Counter plot for gender ratio w.r.t to output")
plt.show()

In [ ]:
plt.figure(figsize=(8,3))
sns.countplot(x='cp',hue='output',data=df)
plt.xticks(ticks=[0,1,2,3], labels=['typical angina', 'atypical angina', 'non-anginal pain', 'asymptomatic'], rotation=0);
plt.title("Various types of chest pains with respect to output")
plt.show()

In [ ]:
##lets just do some data analysis
fig = plt.figure(figsize=(15,25))
rows = 4
cols = 4

for i,col in enumerate(df.columns):
  ax = fig.add_subplot(rows, cols, i+1)

  ## Make the boxplot and the stripplot
  sns.boxplot(x = df[col], ax = ax, palette = 'vlag')
  sns.stripplot(x = df[col], ax = ax, size = 4, linewidth = 0)

In [ ]:
##lets do boxplot analysis only for columns which should be numerical
#so lets first declare the columns which are numerical

num_cols = ['age', 'trtbps', 'chol', 'thalachh', 'oldpeak']
##lets just do some data analysis
fig = plt.figure(figsize=(15,25))
rows = 4
cols = 4

for i,col in enumerate(num_cols):
  ax = fig.add_subplot(rows, cols, i+1)

  ## Make the boxplot and the stripplot
  sns.boxplot(x = df[col], ax = ax, palette = 'vlag')
  sns.stripplot(x = df[col], ax = ax, size = 4, linewidth = 0)

# We can see there are lot of outliers
# So lets do some basic Feature Engineering

In [ ]:
##Because our quantile range is 25 to 75 we set the range.
##Now lets call our function

def outlier(df, col_name, low_quantile=0.25, up_quantile=0.75):
    Q1 = df[col_name].quantile(low_quantile)
    Q3 = df[col_name].quantile(up_quantile)
    IQR = Q3 - Q1
    low_limit = Q1 - 1.5 * IQR
    up_limit = Q3 + 1.5 * IQR
    return low_limit, up_limit

In [ ]:
##Now lets call our function outlier handler

def check_outlier(df, col_name):
    low_limit, up_limit = outlier(df, col_name)
    if df[(df[col_name] > up_limit) | (df[col_name] < low_limit)].any(axis=None):
        return True
    else:
        return False

In [ ]:
for col in num_cols:
    print(col, check_outlier(df, col))

In [ ]:
##Now lets replace the outliers with lower limit and upper limit

def replace_thresholds(df, variable):
    low_limit, up_limit = outlier(df, variable)
    df.loc[(df[variable] < low_limit), variable] = low_limit
    df.loc[(df[variable] > up_limit), variable] = up_limit

In [ ]:
for col in num_cols:
    print(col, replace_thresholds(df, col))

# ONE HOT ENCODER

In [ ]:
def one_hot_encoder(df, categorical_col, drop_first=True):
    dataframe = pd.get_dummies(df, columns=categorical_col, drop_first=drop_first)
    return dataframe

ohe_cols = [col for col in df.columns if 12 >= df[col].nunique() > 2]
df = one_hot_encoder(df, ohe_cols, drop_first=True)

In [ ]:
from sklearn.preprocessing import StandardScaler,LabelEncoder
scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

In [ ]:
df.head()

# MODEL TRAINING

# LOGISTIC REGRESSION

In [ ]:
##libraries declaration

import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import warnings

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
##training our X and Y model

X = df.drop('output', axis=1)
y = df['output']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.3)


In [ ]:
# Initialize the Logistic Regression classifier
clf = LogisticRegression()


In [ ]:
##Hypertuning parameters
parameters = {'penalty': ['l1', 'l2', 'elasticnet', 'none'], 'C': [1, 10, 20]}

In [ ]:
##GRIDSEARCHCV
clf = GridSearchCV(clf, param_grid=parameters, cv=5)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
#Finding our best parameters
best_params = clf.best_params_

In [ ]:
##Throwing it in out our LR
clf = LogisticRegression(C=best_params['C'], penalty=best_params['penalty'])

In [ ]:
##fitting our model
clf.fit(X_train, y_train)

In [ ]:
##printing accuracy
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_pred, y_test)
print(f'Accuracy: {accuracy:.2f}')

# DECISION TREE CLASSIFIER

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [ ]:
#Create a DecisionTreeClassifier
treeclassifier = DecisionTreeClassifier()

# Define hyperparameters for grid search
parameters = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': [1, 2, 3, 4, 5],
    'max_features': ['auto', 'sqrt', 'log2']
}

In [ ]:
#prepare our model for GRIDSEARCHCV
clf1 = GridSearchCV(treeclassifier, param_grid=parameters, cv=5, scoring='accuracy')

In [ ]:
#Fitting out model
clf1.fit(X_train, y_train)


In [ ]:
#Finding the best parameters
best_params = clf1.best_params_

In [ ]:
##Training
clf1 = DecisionTreeClassifier(**best_params)

In [ ]:
##Fiting our model
clf1.fit(X_train, y_train)

In [ ]:
##Predicting our model
y_pred1 = clf1.predict(X_test)


In [ ]:
##Showing accuracy

accuracy1 = accuracy_score(y_pred1, y_test)
print(f'Accuracy: {accuracy1:.2f}')

# RANDOM FOREST CLASSIFIER

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score


In [ ]:
# Create a RandomForestClassifier
clf2 = RandomForestClassifier()

# Define hyperparameters for random search
params = {
    'max_depth': [3, 5, 10, None],  # 'None' should not be a string
    'n_estimators': [100, 200, 300],
    'criterion': ['gini', 'entropy']
}

In [ ]:
##Doing Randomized search cv
clf2 = RandomizedSearchCV(clf2, param_distributions=params, scoring='accuracy', cv=5, verbose=3, n_iter=10)

In [ ]:
##Fitting our model
clf2.fit(X_train, y_train)

In [ ]:
# Get the best hyperparameters found by random search
best_params = cv.best_params_

In [ ]:
##Declaring our model
clf2 = RandomForestClassifier(**best_params)

In [ ]:
# Fit the best forest classifier to your training data
clf2.fit(X_train, y_train)

In [ ]:
##Predicting our model
y_pred2 = clf2.predict(X_test)

In [ ]:
##Showing accuracy
accuracy2 = accuracy_score(y_pred2, y_test)
print(f'Best Parameters: {best_params}')
print(f'Accuracy: {accuracy2:.2f}')

# XGBOOST

In [ ]:
import xgboost as xb

##Classifying our model
clf3 = xb.XGBClassifier()

##Fitting our model
clf3.fit(X_train,y_train)

In [ ]:
##Predicting Model

y_pred3 = clf3.predict(X_test)

In [ ]:
##Showing our Accuracy
accuracy3 = accuracy_score(y_pred3,y_test)
print(f'Accuracy: {accuracy3:.2f}')

# ADABOOST, GRADIENT BOOSTING CLASSIFIER

In [ ]:

from sklearn.ensemble import AdaBoostClassifier,GradientBoostingClassifier

In [ ]:
##Declaring our model
clf4 = AdaBoostClassifier()

In [ ]:
##Fitting our model

clf4.fit(X_train,y_train)

In [ ]:
#Prediction

y_pred4 = clf4.predict(X_test)

In [ ]:
##Accuracy for ada boost
accuracy4 = accuracy_score(y_pred4,y_test)
print(f'Accuracy: {accuracy4:.2f}')

In [ ]:
##Classifying our model

clf5 = GradientBoostingClassifier()

In [ ]:
##Fitting our model

clf5.fit(X_train,y_train)

In [ ]:
##Predicting our output

y_pred5 = clf5.predict(X_test)

In [ ]:
##Accuracy
accuracy5 = accuracy_score(y_pred5,y_test)

print(f'Accuracy: {accuracy5:.2f}')

In [ ]:
print(f'LogisticRegression : {accuracy:.2f}')
print(f'DecisionTreeClassifer : {accuracy1:.2f}')
print(f'RandomForestClassifier : {accuracy2:.2f}')
print(f'XGBoost : {accuracy3:.2f}')
print(f'ADABoost : {accuracy4:.2f}')
print(f'GradientBoosting : {accuracy5:.2f}')

In [ ]:
##For this dataset,I was able to do 6 ML Models
##We can check various algorithms as well. 
## We can also do a little more better data handling.
##If you think anything is wrong, please let me know.
## If you think you have better suggestions do let me know.
##And if you like this, please upvote.
##Thank you everyone & have a nice day :)